## Extract Text from PDFs

In [3]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Example usage
pdf_text = extract_text_from_pdf(r"C:\Users\Farrukh\Downloads\Vol-4_Introduction_to_Technical_Analysis.pdf")
with open("trading_corpus.txt", "w") as f:
    f.write(pdf_text)

## Fine-Tune LLaMA

In [4]:
from transformers import LlamaTokenizer, LlamaForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset

# Load tokenizer and model
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Replace with the model you have access to
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

# Load and tokenize dataset
data_path = r"C:\Users\Farrukh\Downloads\fine_tune_dataset.txt"  # Your dataset
dataset = load_dataset("text", data_files=data_path)["train"]

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./llama_finetuned",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./llama_finetuned")
tokenizer.save_pretrained("./llama_finetuned")


RuntimeError: Failed to import transformers.models.llama.modeling_llama because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject